In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [21]:
# Load the data from the CSV file
# df = pd.read_csv('train_data_mod.csv')
# df = pd.read_csv('Hotel_Reservations_mod.csv')
# df = pd.read_csv('originalData/train_data.csv')
# df = pd.read_csv('originalData/Hotel_Reservations.csv')
df = pd.read_csv('originalData/train_data.csv')

df2 = pd.read_csv('originalData/test_data.csv')


In [22]:
# Scale the numerical variables
scaler = StandardScaler()
df[['LeadTime', 'NumWeekendNights', 'NumWeekNights', 'NumAdults', 'NumChildren', 
    'RepeatedGuest', 'NumPrevCancellations', 'NumPreviousNonCancelled', 
    'AvgRoomPrice', 'SpecialRequests']] = scaler.fit_transform(df[['LeadTime', 'NumWeekendNights', 
                                                                     'NumWeekNights', 'NumAdults', 'NumChildren', 
                                                                     'RepeatedGuest', 'NumPrevCancellations', 
                                                                     'NumPreviousNonCancelled', 'AvgRoomPrice', 
                                                                     'SpecialRequests']])

In [23]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = ['MealPlan', 'Parking', 'RoomType', 'MarketSegment', 'BookingStatus']
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    
    df2[col] = le.fit_transform(df2[col])
    
    # df2[col] = le.fit_transform(df2[col])
df

,BookingID,LeadTime,ArrivalYear,ArrivalMonth,ArrivalDate,NumWeekendNights,NumWeekNights,MealPlan,Parking,RoomType,NumAdults,NumChildren,MarketSegment,RepeatedGuest,NumPrevCancellations,NumPreviousNonCancelled,AvgRoomPrice,SpecialRequests,BookingStatus
0,1,-0.876476,2018,3,31,-0.930543,-0.849502,0,0,0,-1.633471,-0.259243,2,-0.16221,-0.064378,-0.087379,-0.241763,-0.788557,0
1,2,0.357733,2018,2,28,1.359612,-0.849502,0,0,0,-1.633471,-0.259243,4,-0.16221,-0.064378,-0.087379,-1.211752,-0.788557,0
2,3,-0.864832,2018,7,25,0.214534,-0.146148,0,0,0,0.295297,2.222853,4,-0.16221,-0.064378,-0.087379,0.749623,0.482631,1
3,4,-0.957980,2017,9,12,-0.930543,-0.849502,0,0,0,0.295297,-0.259243,4,-0.16221,-0.064378,-0.087379,1.384395,3.025008,1
4,5,-0.666893,2018,3,7,0.214534,0.557205,0,0,0,0.295297,-0.259243,3,-0.16221,-0.064378,-0.087379,-0.469996,-0.788557,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29015,29016,-0.969624,2018,11,21,0.214534,-0.146148,0,0,0,-1.633471,-0.259243,4,-0.16221,-0.064378,-0.087379,-0.080003,0.482631,1
29016,29017,-0.783328,2018,4,12,-0.930543,-0.146148,3,0,0,0.295297,-0.259243,4,-0.16221,-0.064378,-0.087379,0.442935,0.482631,1
29017,29018,-0.899763,2017,9,29,-0.930543,0.557205,0,0,0,-1.633471,-0.259243,2,-0.16221,-0.064378,-0.087379,-1.097635,-0.788557,1
29018,29019,-0.760041,2018,9,18,1.359612,-0.146148,0,0,0,-1.633471,-0.259243,3,-0.16221,-0.064378,-0.087379,-0.384408,-0.788557,0


In [24]:
# X_train, X_test, y_train, y_test = train_test_split(df.drop(['BookingStatus','BookingID', 'NumWeekendNights', 'NumWeekNights','NumAdults','NumChildren','AvgRoomPrice','ArrivalMonth','ArrivalDate','ArrivalYear' ], axis=1), df['BookingStatus'], 
#                                                     test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(df.drop(['BookingStatus','BookingID'], axis=1), df['BookingStatus'], 
                                                    test_size=0.2, random_state=42)


In [32]:
# Train the logistic regression model
# lr = LogisticRegression(random_state=42)
lr = RandomForestClassifier(random_state=42)
# lr = LinearRegression()
# lr = DecisionTreeClassifier(random_state=42)
# lr.fit(X_train, y_train)
lr.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [33]:
# from sklearn.feature_selection import RFECV
# print(logreg.coef_)

# # Method 2: Recursive feature elimination
# selector = RFECV(logreg, cv=5)
# selector.fit(X, y)
# print(selector.support_)

# # Method 3: L1 regularization
# logreg_l1 = LogisticRegression(penalty='l1', solver='liblinear')
# logreg_l1.fit(X, y)
# print(logreg_l1.coef_)

In [34]:
# Evaluate the performance of the model
y_pred = lr.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1 Score:', f1_score(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))


Accuracy: 0.9062715368711234
Precision: 0.9155972359328727
Recall: 0.9483640081799591
F1 Score: 0.9316926167754896
Confusion Matrix:
 [[1550  342]
 [ 202 3710]]


In [28]:
importance = lr.feature_importances_

# Sort feature importance in descending order
sorted_importance = sorted(zip(importance, X_train.columns), reverse=True)

# Print feature importance in descending order
for i, f in enumerate(sorted_importance):
    print("{:2d}) {:<30} {:.2%}".format(i + 1, f[1], f[0]))

 1) LeadTime                       32.13%
 2) AvgRoomPrice                   16.17%
 3) SpecialRequests                9.81%
 4) ArrivalDate                    9.19%
 5) ArrivalMonth                   8.11%
 6) MarketSegment                  5.43%
 7) NumWeekNights                  5.26%
 8) NumWeekendNights               3.74%
 9) ArrivalYear                    2.60%
10) NumAdults                      2.49%
11) MealPlan                       1.76%
12) RoomType                       1.63%
13) NumChildren                    0.75%
14) Parking                        0.58%
15) RepeatedGuest                  0.19%
16) NumPreviousNonCancelled        0.14%
17) NumPrevCancellations           0.03%


In [29]:
# from sklearn.feature_selection import RFE
# n_features_to_select = 5

# # Use RFE algorithm to select best subset of features
# rfe = RFE(estimator=lr, n_features_to_select=n_features_to_select)
# rfe.fit(X_train, y_train)

# # Get selected features
# selected_features = X_train.columns[rfe.support_]

# # Evaluate performance of random forest classifier using selected features
# lr.fit(X_train[selected_features], y_train)
# score = lr.score(X_test[selected_features], y_test)

# print(f'Selected Features: {selected_features}')
# print(f'Test Set Score: {score:.3f}')

In [30]:
missing_features = set(X_train.columns) - set(df2.columns)

if len(missing_features) > 0:
    missing_indices = pd.DataFrame(0, index=df2.index, columns=missing_features)
    df2 = pd.concat([df2, missing_indices], axis=1)


In [31]:
X_df2 = df2.drop(['BookingID', 'BookingStatus'], axis=1)
predicted_booking_status = lr.predict(X_df2)
df2['BookingStatus'] = predicted_booking_status

df2.to_csv('team4.csv', index=False)

# print(X_df2.shape)
# print(X_train.shape)